*Airflow DAG's*

In [43]:
#Step 1: Import modules
import msvcrt
from datetime import datetime
from datetime import timedelta
import airflow
from airflow import DAG
from airflow.operators.bash import BashOperator
#from airflow import python
#from airflow.operators import python
#from airflow.operators.python_operator import PythonOperator

In [44]:
#Step 2: Default Args
default_args= {
    'owner': 'airflow',
    'start_date': airflow.utils.dates.days_ago(2),
    'depends_on_past' : False,
    'email' : ['airflow@example.com'],
    'email_on_failure' : False,
    'email_on_retry' : False,
    'retries': 1,
    'retry_delay':timedelta(minutes=5),
}

In [45]:
#Step 3: Instantiate a DAG
with DAG("my_dag", start_date=datetime(2021, 6, 5),
    schedule_interval="@daily", catchup=False) as dag:

        Connection_sql = BashOperator(
            task_id="Connection_sql",
            bash_command=Connection_sql
        )
        
        Connection_mysql = BashOperator(
            task_id="Connection_mysql",
            bash_command=Connection_mysql
        )

In [46]:
#Step 4: Set up tasks
import pypyodbc # pip install pypyodbc
import pandas as pd # pip install pandas

def Connection_sql():
    SERVER_NAME = 'DESKTOP-H2M6ONO'
    DATABASE_NAME = 'test'

    conn = pypyodbc.connect("""
        Driver={{SQL Server Native Client 11.0}};
        Server={0};
        Database={1};
        Trusted_Connection=yes;""".format(SERVER_NAME, DATABASE_NAME)
    )

    sql_query1 = """
    SELECT * FROM organization_provider WITH (NoLock)
    """

    # With Headers
    organization_provider = pd.read_sql(sql_query1, conn)
    
def Connection_mysql():
    from mysql.connector import (connection)
    cnx = connection.MySQLConnection(user='root', password='root',
                                 host='localhost',
                                 database='medical_data',
                                auth_plugin='mysql_native_password')

    sql_query1 = 'Select * from encounters';
    encounters = pd.read_sql(sql_query1, cnx)
    cnx.close()

In [48]:
#Step 5: Set up dependencies like upstream & downstream

Connection_sql>>Connection_mysql

TypeError: unsupported operand type(s) for >>: 'function' and 'function'

In [49]:
import datetime as dt

from airflow import DAG
from airflow.operators.bash_operator import BashOperator
#from airflow.operators.python_operator import PythonOperator


def print_world():
    print('world')


default_args = {
    'owner': 'me',
    'start_date': dt.datetime(2017, 6, 1),
    'retries': 1,
    'retry_delay': dt.timedelta(minutes=5),
}


with DAG('airflow_tutorial_v01',
         default_args=default_args,
         schedule_interval='0 * * * *',
         ) as dag:

    print_hello = BashOperator(task_id='print_hello',
                               bash_command='echo "hello"')
    sleep = BashOperator(task_id='sleep',
                         bash_command='sleep 5')
    print_world = BashOperator(task_id='print_world',
                                 bash_command=print_world)


print_hello >> sleep >> print_world

<Task(BashOperator): print_world>